### Analyzing New York City taxi data using big data tools

#### 1. The NYC taxi data

In [1]:
import arcgis
from arcgis.gis import GIS

ago_gis = GIS() # Connect to ArcGIS Online as an anonymous user
search_subset = ago_gis.content.search("NYC_taxi_subset", item_type = "Feature Layer")
subset_item = search_subset[0]
subset_item

<Item title:"NYC taxi subset" type:Feature Layer Collection owner:api_data_owner>

In [2]:
# Let us bring up a map to display the data.
subset_map = ago_gis.map("New York, NY", zoomlevel=11)
subset_map

MapView(layout=Layout(height='400px', width='100%'))